In [1]:
import json
# Open and read the JSON file
with open('./divided.json', 'r') as file:
    data = json.load(file)
    
data[list(data.keys())[2]] = data[list(data.keys())[2]][:-90]
_ = data.pop('كتاب رسائل ابن حزم'), data.pop('كتاب الحيوان')

In [5]:
def prepare_text(text):
    import re
    text = re.sub(r'[\[\(<«](\d+)[\]\)>»]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [6]:
import pandas as pd
df = pd.DataFrame(columns=['text', 'book'])

for title in list(data.keys()):
    df1 = pd.DataFrame()
    texts = data[title]
    
    temp = pd.DataFrame(texts)
    
    str_ = ' '.join(temp.astype(str).values.flatten())
    
    temp = pd.DataFrame([str_])
    
    df1['text'] = temp[0].str.replace(r'((?<!\.)\.(?!\.))|(؟)', r'\1\2#', regex=True).str.split(r"#").explode()
    df1['text'] = df1['text'].map(prepare_text)
    
    df1['book'] = title
    
    df = pd.concat([df, df1])

df = df[df['text'].str.len() > 6]
df = df[~df['text'].str.contains('\.\.\.')]
df.reset_index(inplace=True, drop=True)

In [15]:
df

,text,book
0,[باب في الدخول على الملوك] [دخول الأشراف] إن ك...,كتاب التاج في أخلاق الملوك
1,فإن استدناه، قرب منه، فأكب على أطرافه يقبلها، ...,كتاب التاج في أخلاق الملوك
2,فإن أومأ إليه بالقعود، قعد.,كتاب التاج في أخلاق الملوك
3,فإن كلمه، أجابه بانخفاض صوتٍ، وقلة حركة.,كتاب التاج في أخلاق الملوك
4,وإن سكت، نهض من ساعته، قبل أن يتمكن به مجلسه ب...,كتاب التاج في أخلاق الملوك
...,...,...
13565,وأما في الحقيقة فذلك إنما هو بإرادة من لا يعتر...,كتاب شرح تشريح القانون لابن سينا
13566,كمل ذلك ولله الحمد والمنّة ونسأله السّلامة في ...,كتاب شرح تشريح القانون لابن سينا
13567,النسخة م: آخره ولله الحمد والمنّة وصلى الله عل...,كتاب شرح تشريح القانون لابن سينا
13568,تمت في تاريخ شهر ذي القعدة الحرام بعد ما خلت ث...,كتاب شرح تشريح القانون لابن سينا


In [19]:
df.loc[:, 'text'].to_excel('Arabic.xlsx')

In [25]:
df_en = pd.read_excel("./en.xlsx").drop(columns=['Unnamed: 0'])

In [32]:
df['en'] = df_en['text']
df['ar'] = df['text']
df = df.drop(columns=['text'])

In [42]:
df.sample().values

array([['كتاب التاج في أخلاق الملوك',
        ' So he asked permission, and the chamberlain said to him: This is not the time to give permission to the prince.',
        'فاستأذنن فقال له الحاجب: ليس هذا بوقت إذنٍ على الأمير.']],
      dtype=object)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='REDACTED')
import random

for i in range(5):
    sample = df.loc[random.randint(1, 12000), 'text']
    
    completion2 = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"""
Arabic: {sample}
English:
            """}
        ]
    )
    
    print(sample)
    print(completion2.choices[0].message.content)
    print("-"*50)

In [44]:
df.to_csv("Finished.csv")